For a given antenna size, find the solutions with minimal ambiguity
the doppler bandwidth can be reduced

1. find the optimal line in the timing diagram plane (based on antenna width), i.e., elevation 3-db angle
2. find all valid swaths close to the optimal line in the timing diagram plane. don't care about the nadir return, this can be discarded in the final image anywais.
3. make the valid swaths selectable by an order *n* identifying how far the point is from the optimal.


plot timing diag
plot optimal prf vs looking angle
todo test timingdiagram plotter here

In [40]:
# From Michelangelo Villano's hands on lab
# To open separate plot-windows outside the browser uncomment one of the following two lines
%matplotlib qt
#get_ipython().run_line_magic('matplotlib','qt5')

# To open a Plot-window within notebook with zoom/edit control uncomment one of the following two lines
# %matplotlib notebook
# get_ipython().run_line_magic('matplotlib','notebook')

# options are 'osx', 'qt4', 'qt5', 'gtk3', 'wx', 'qt', 'gtk', 'tk' , 'notebook' , 'inline'

In [41]:
## Dependencies
from libs.timing_diagram import *
from libs.design_functions import *
import numpy as np
import matplotlib.pyplot as plt

In [42]:
# constants
c = 299792458

# radar parameters
dutycycle = 0.25  # duty cycle
h = 500e3  # height
wavelength = c / 10e9  # wavelength

# satellite speed
vs = orbital_speed(h)

# antenna size
Wa = 0.3  # antenna width in meters
La = 2  # antenna length in meters

# nadir duration in fractions of PRI for visualization in timing diagram
# note, it makes sense to use this fractional quantity as the nadir duration, if unfocused (e.g. saturated receiver), will be proportional to the impulse on time.
nadir_duration = 2 * dutycycle


In [43]:
# PRF axis
PRI = 1 / 7050
prf = np.linspace(1 / PRI - 4000, 1 / PRI + 4000, 100)

# plot
fig, ax = plt.subplots(1)
time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False)
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h)
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)

C:\Users\smen851\PycharmProjects\systemtradeoffs\libs\design_functions.py:42: RuntimeWarning: invalid value encountered in arccos
  beta = arccos(((re + h) ** 2 - re ** 2 + rs ** 2) / (2 * (re + h) * rs))
C:\Users\smen851\PycharmProjects\systemtradeoffs\libs\design_functions.py:43: RuntimeWarning: invalid value encountered in arccos
  alpha = arccos(((re + h) ** 2 + re ** 2 - rs ** 2) / (2 * (re + h) * re))


(100.0, 300.0)

Finding the "optimal" PRF for each looking angle at each PRF -- Ground range broadside point
1. find the incidence angle from ground range
2. find the min max incidence angles of the swath extremes $\eta_{1,2} = \eta_i \pm \dfrac{\lambda}{W_A}$
3. find the slant range delta from $\eta_{1,2}$
4. find PRF from delta range

In [44]:
# 1 ground range axis in m
ground_range = np.linspace(0, 2000, 500) * 1000
# 2 incidence angle
slant_range = range_ground_to_slant(ground_range, h)
# loopy but functional
ground_range, eta = range_slant_to_ground(slant_range, h)  # the incidence angle is the second parameter returned
# 3 incidence range
eta1 = eta - wavelength / (2 * Wa)
eta2 = eta + wavelength / (2 * Wa)
slant_range_1, ground_range_1 = range_from_theta(eta1 * 180 / np.pi, h)
slant_range_2, ground_range_2 = range_from_theta(eta2 * 180 / np.pi, h)
# 4 slant range delta
slant_delta_range = slant_range_2 - slant_range_1
# 5 prf from slant range
pri = 2 * slant_delta_range / c
prf_opt = 1 / pri

C:\Users\smen851\AppData\Local\Temp\ipykernel_15828\1203726107.py:16: RuntimeWarning: divide by zero encountered in divide
  prf_opt = 1 / pri


In [45]:
# Add to the plot
ax.plot(prf_opt, ground_range / 1000, 'red')

plt.show()

In [46]:
sr = range_ground_to_slant(200e3, h)
# loopy but functional
gr, eta = range_slant_to_ground(sr, h)  # the incidence angle is the second parameter returned
print(eta * 180 / np.pi, gr / 1000)

23.4734561406255 199.99999999999412


Find the minimum PRF line, i.e. the Doppler bandwidth approximated as
\begin{equation}
    B_D = \dfrac{2}{La} v_g
\end{equation}
with the ground velocity
\begin{equation}
 v_g = \dfrac{R_E \cos\Theta_E}{ R_E + h} v_s
\end{equation}
where
\begin{equation}
    \Theta_E = \dfrac{r_g}{R_E}
\end{equation}
$R_E$ is the Earth radius, $r_g$ the ground range, $v_s$ the satellite orbital speed


In [47]:
prf_min = 2 * ground_speed(ground_range, vs, h) / La

In [48]:
# Add to the plot
ax.plot(prf_min, ground_range / 1000, 'green')

plt.show()

In [49]:
range_from_theta(26.8)

(555062.2974338281, 232104.76792566918)

Find the swath given a point in the time diagram and plot it as a vetical line
i.e.
find closest end of transmission
fing closest start of transmission
plot a line (constant prf)

In [50]:
rg_coordinate = 200e3 # m
prf_coordinate = 6780 # Hz
swath_prf = np.zeros(2)
swath_rg = np.zeros_like(swath_prf)
swath_rg[0], swath_prf[0] = last_end_of_transmission(rg_coordinate, prf_coordinate, dutycycle, h)
swath_rg[1], swath_prf[1] = next_start_of_transmission(rg_coordinate, prf_coordinate,dutycycle, h)

In [51]:
# total swath considering compression time
# 1 swath from pulse to pulse
swath_rs = range_ground_to_slant(swath_rg, h)
# 2 swath without 1 pulse period (half on each side)
swath_rs_compressed = swath_rs + np.array([(1 / swath_prf[0] * dutycycle * c / 4), - 1 / swath_prf[1] * dutycycle * c / 4])
# 3 convert back to ground range
swath_rg_compressed, swath_rg_eta_compressed = range_slant_to_ground(swath_rs_compressed, h)
# swath width in km
print('swath width on ground =', (swath_rg_compressed[-1] - swath_rg_compressed[0]) / 1000, 'km' )

swath width on ground = 27.73750754106915 km


In [52]:
# add to plot
ax.plot(swath_prf, swath_rg / 1000 , 'k')
ax.plot(swath_prf, swath_rg_compressed / 1000, 'P', color='k')
plt.show()